In [ ]:
# importa os pacotes necessários
import numpy as np
import os, gc, cv2, random
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 

import warnings
warnings.filterwarnings("ignore")

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
import sklearn.metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import  cross_val_score

from sklearn import svm
from skimage.feature import greycomatrix, greycoprops
from skimage.feature import hog
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

In [ ]:
df = pd.read_csv('kaggle/data_pca.csv')
df_labels = pd.read_csv('kaggle/data_labels.csv')

In [ ]:
#df=(df-df.min())/(df.max()-df.min())

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df, df_labels.label.values, test_size = 0.3, random_state = 42)
del df
X_train.shape,X_test.shape

# Logistic Regression
Utilizando regressão logistica para ter uma base de comparação com os demais modelos.

#### nomalizado antes e padding

In [ ]:
%%time
logreg = LogisticRegression(solver='saga',n_jobs=2)
logreg.fit(X_train, y_train)

In [ ]:
# avalia predicao
y_pred = logreg.predict(X_test)
report = classification_report(y_test, y_pred,output_dict = True)
pd.DataFrame(report)

In [ ]:
%%time
logreg = LogisticRegression(solver='saga',n_jobs=4)
logreg.fit(X_train.iloc[:,:1500], y_train)

In [ ]:
# avalia predicao
y_pred = logreg.predict(X_test.iloc[:,:1500])
report = classification_report(y_test, y_pred,output_dict = True)
pd.DataFrame(report)

# SVM

In [ ]:
%%time
clf = svm.SVC(gamma='scale',verbose=True)
clf.fit(X_train, y_train)  

In [ ]:
y_pred = clf.predict(X_test)
report = classification_report(y_test, y_pred,output_dict = True)
pd.DataFrame(report)